In [38]:
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.nn as nn


In [39]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU available:", device)
    torch.cuda.init()
else:
    device = torch.device("cpu")
    print("GPU unavailable: CPU")

GPU unavailable: CPU


In [40]:
class Sales:
    def __init__(self, all_offer_sets, sell_num, mask):
        self.offer_set_list = []
        self.fw = None
        self.N_sales = torch.tensor(np.concatenate(sell_num, axis=1), dtype=torch.float64, device=device)
        self.offer_feature = torch.tensor(all_offer_sets, dtype=torch.float64)
        self.original_shape = self.offer_feature.shape
        self.feature_concat = self.offer_feature.reshape(-1, self.offer_feature.shape[2]).to(device)
        self.N = len(all_offer_sets)
        self.mask = torch.tensor(mask, dtype=torch.float64, device=device)
        self.mask_flat = self.mask.reshape((-1,))
        self.masked_feature_concat = self.feature_concat[self.mask_flat == 1]

    def calculate_all_choice_prob(self, W):
        rule_feature = self.feature_concat
        self.fw = self.calculate_choice_prob(rule_feature, W)
        return self.fw

    def calculate_choice_prob(self, rule_feature, W):
        Z = W(rule_feature).reshape(self.original_shape[:2])
        masked_Z = Z.masked_fill(self.mask == 0, float('-inf'))
        softmax_result = F.softmax(masked_Z, dim=-1).reshape((-1, 1))[self.mask_flat == 1]
        return softmax_result

In [41]:
class Deep_MNL(nn.Module):
    def __init__(self, input_dim):
        super(Deep_MNL, self).__init__()
        self.layer1 = nn.Linear(input_dim, 100, dtype=torch.float64)
        self.layer2 = nn.Linear(100, 30, dtype=torch.float64)
        self.layer3 = nn.Linear(30, 1, dtype=torch.float64)
        # self._initialize_weights()

    def _initialize_weights(self):
        nn.init.constant_(self.layer1.weight, 0)
        nn.init.constant_(self.layer1.bias, 0)
        nn.init.constant_(self.layer2.weight, 0)
        nn.init.constant_(self.layer2.bias, 0)
        nn.init.constant_(self.layer3.weight, 0)
        nn.init.constant_(self.layer3.bias, 0)

    def forward(self, x):
        x = self.layer1(x)
        x = torch.relu(x)
        x = self.layer2(x)
        x = torch.relu(x)
        x = self.layer3(x)
        return x

In [42]:
class Problem_MNL:
    def __init__(self, S, N, M): #Option set, Number of sales, Mask
        self.W = None
        self.sales = Sales(S, N, M)
        # Define and initialize the ruleset
        self.feature_num = S.shape[-1]
        # Put the feature to the GPU
        self.sales.feature_concat = self.sales.feature_concat.to(device)
        # Define a consumer list that contain consumer types
        self.consumer_list = []
        # Define the main problem NLL loss
        self.NLL_main = None
        # Define the current likelihood convex combination
        self.g = None
        # Define the current likelihood gradient for support finding
        self.NLL_gradient = None
        # Define a list to contain all fw choice likelihood
        self.fw_list = []
        # Define a new sales data for further estimation
        self.sales_estimate = None

    def loss(self, W):
        N = self.sales.N
        N_sales = self.sales.N_sales
        normalize_term = torch.tensor(1 / N, dtype=torch.float64, device=device)
        fw = self.sales.calculate_all_choice_prob(W)
        fw_log = torch.log(fw)
        return -normalize_term * torch.matmul(N_sales, fw_log)

    def optimize_w(self):
        encoder = Deep_MNL(14)
        loss_previous = 1e10000
        optimizer = optim.Adam(encoder.parameters(), lr=5e-6)
        optimizer.zero_grad()
        NLL_list = []
        early_stop_count = 0
        epoches = 2000
        for epoch in range(epoches):
            LOSS = self.loss(encoder)
            LOSS.backward()
            optimizer.step()
            if (epoch + 1) % 10 == 0:
                print(f'Epoch [{epoch + 1}/{epoches}], Loss: {LOSS.item():.4f}')
            loss_current = LOSS.item()
            # if loss_current < loss_previous:
            #     loss_previous = loss_current
            # else:
            #     early_stop_count += 1
            #     if early_stop_count > 0:
            #         break
            NLL_list.append(loss_current)
        self.W = encoder
        return NLL_list

    def estimate(self, S, N, M):
        sales_estimate = Sales(S, N, M)
        N = sales_estimate.N
        N_sales = sales_estimate.N_sales
        normalize_term = torch.tensor(1 / N, dtype=torch.float64, device=device)
        fw = sales_estimate.calculate_all_choice_prob(self.W)
        fw_log = torch.log(fw)
        return -normalize_term * torch.matmul(N_sales, fw_log)

In [43]:
def get_offer_data(data_para):
    offerset_list = []
    sell_list = []
    mask_list = []
    max_num = 33
    for srch_id, group in data_para:
        num_product = len(group)
        offerset = group.drop(columns=['booking_bool', 'srch_id']).values
        offer_dummy = np.zeros((max_num - num_product, offerset.shape[1]))
        offerset = np.vstack((offerset, offer_dummy)) #Padding Offer Sets
        offer_mask = np.append(np.ones(num_product + 1), np.zeros(max_num - num_product - 1)) # Mask for 0 from padding

        if group['booking_bool'].sum() == 0: # No hotel reservations
            num_sell = np.append(group['booking_bool'].values, 1).reshape(1, -1) # Create and Select "not buy" option
        else:
            num_sell = np.append(group['booking_bool'].values, 0).reshape(1, -1)

        offerset_list.append(offerset)
        sell_list.append(num_sell)
        mask_list.append(offer_mask)

    offerset_list = np.array(offerset_list)
    mask_list = np.array(mask_list)
    return offerset_list, sell_list, mask_list


search_info = ['srch_id']
feature = ['position', 'prop_starrating',
           'prop_location_score1', 'prop_log_historical_price',
           'prop_brand_bool', 'promotion_flag', 'srch_booking_window', 'srch_length_of_stay',
           'srch_adults_count', 'srch_children_count', 'srch_room_count',
           'srch_saturday_night_bool', 'price_usd', 'random_bool', 'booking_bool']

tr_data = pd.read_csv('train_28-32_10000.csv')
te_data = pd.read_csv('test_28-32_1000.csv')
tr_data = tr_data[search_info + feature]
te_data = te_data[search_info + feature]
tr_offerset_list, tr_sell_list, tr_mask_list = get_offer_data(tr_data.groupby('srch_id'))
te_offerset_list, te_sell_list, te_mask_list = get_offer_data(te_data.groupby('srch_id'))

In [ ]:
def train_MNL():
    problem = Problem_MNL(tr_offerset_list, tr_sell_list, tr_mask_list)
    NLL_list = problem.optimize_w()
    return problem, NLL_list


MNL_problem, NLL_LIST = train_MNL()
plt.figure(figsize=(10, 6))
plt.plot(NLL_LIST, marker='o', linestyle='-', color='b', label='MNL')
plt.title('NLL LOSS VALUE')
plt.xlabel('ITERATION')
plt.ylabel('NLL LOSS')
plt.legend()
plt.grid(True)
plt.show()

Epoch [10/2000], Loss: 4.9318
Epoch [20/2000], Loss: 4.8725
Epoch [30/2000], Loss: 4.8077
Epoch [40/2000], Loss: 4.7389
Epoch [50/2000], Loss: 4.6676
Epoch [60/2000], Loss: 4.5952
Epoch [70/2000], Loss: 4.5227
Epoch [80/2000], Loss: 4.4511
Epoch [90/2000], Loss: 4.3807
Epoch [100/2000], Loss: 4.3120
Epoch [110/2000], Loss: 4.2453
Epoch [120/2000], Loss: 4.1808
Epoch [130/2000], Loss: 4.1185
Epoch [140/2000], Loss: 4.0587
Epoch [150/2000], Loss: 4.0015


In [ ]:
print('Test Loss: ', MNL_problem.estimate(te_offerset_list, te_sell_list, te_mask_list))
print('Train Loss: ', MNL_problem.estimate(tr_offerset_list, tr_sell_list, tr_mask_list))